In [2]:
from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta
from sqlalchemy import or_

# Configuration de l'application
app = Flask('COP1_du_13')
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.secret_key = 'supersecretkey'
db = SQLAlchemy(app)  # Base de données pour les bénévoles et missions

# Classe pour les bénévoles
class Benevole(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nom = db.Column(db.String(100), nullable=False)
    prenom = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    telephone = db.Column(db.String(20), nullable=False)

# Classe pour les missions
class Mission(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    titre = db.Column(db.String(100), nullable=False)
    date = db.Column(db.String(20), nullable=False)
    heure = db.Column(db.String(10), nullable=False)
    nb_places = db.Column(db.Integer, nullable=False)

# Classe pour les inscriptions des bénévoles aux missions
class Inscription(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    benevole_id = db.Column(db.Integer, db.ForeignKey('benevole.id'), nullable=False)
    mission_id = db.Column(db.Integer, db.ForeignKey('mission.id'), nullable=False)
    date_inscription = db.Column(db.DateTime, default=datetime.utcnow)

In [4]:
# Fonction pour importer les bénévoles depuis un fichier Excel
file_path = '~/Desktop/cop1_app/upload.xlsx/benevole.xlsx' 
df = pd.read_excel(file_path)
df_benevoles = df[['Prénom', 'Nom ', 'Numéro de Téléphone', 'Ton adresse mail ?']]
df_benevoles.columns = ['prenom', 'nom', 'telephone', 'email']

In [6]:
def importer_benevoles():
    for _, row in df_benevoles.iterrows():
        if pd.notna(row['nom']) and pd.notna(row['prenom']):
            nom = str(row['nom']).strip()
            prenom = str(row['prenom']).strip()
            email = str(row['email']).strip() if pd.notna(row['email']) else None
            telephone = None

            if pd.notna(row['telephone']):  
                try:
                    telephone = str(int(row['telephone']))  
                except ValueError:
                    telephone = None  

            if not telephone:
                continue  

            conditions = []
            if email:
                conditions.append(Benevole.email == email)
            if telephone:
                conditions.append(Benevole.telephone == telephone)

            exists = Benevole.query.filter(or_(*conditions)).first() if conditions else None

            if not exists:
                benevole = Benevole(
                    prenom=prenom,
                    nom=nom,
                    email=email,
                    telephone=telephone
                )
                db.session.add(benevole)

    db.session.commit()

# Fonction d'envoi d'email
def envoyer_email(destinataire, sujet, message):
    expediteur = "tonemail@gmail.com"
    mot_de_passe = "motdepasse"
    msg = MIMEText(message)
    msg["Subject"] = sujet
    msg["From"] = expediteur
    msg["To"] = destinataire

    try:
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server.login(expediteur, mot_de_passe)
        server.sendmail(expediteur, destinataire, msg.as_string())
        server.quit()
        print(f"📧 Email envoyé à {destinataire}")
    except Exception as e:
        print(f"⚠️ Erreur lors de l'envoi de l'email : {e}")

In [8]:
# Route pour la page d'accueil
@app.route('/')
def index():
    return render_template('index.html')

# Route pour la liste des bénévoles
@app.route('/benevoles')
def benevoles():
    benevoles = Benevole.query.all()
    return render_template('benevoles.html', benevoles=benevoles)

# Route pour afficher les missions et inscrire des bénévoles
@app.route('/missions', methods=['GET', 'POST'])
def missions():
    missions = Mission.query.all()
    benevoles = Benevole.query.all()

    if request.method == 'POST':
        benevole_id = request.form['benevole_id']
        mission_id = request.form['mission_id']

        mission = Mission.query.get(mission_id)
        if mission and mission.nb_places > 0:
            inscription = Inscription(benevole_id=benevole_id, mission_id=mission_id)
            db.session.add(inscription)
            mission.nb_places -= 1
            db.session.commit()
            flash('✅ Inscription réussie !')
        else:
            flash('❌ Plus de place disponible pour cette mission.')

        return redirect(url_for('missions'))

    return render_template('task.html', missions=missions, benevoles=benevoles)

# Route pour afficher le classement des bénévoles
@app.route('/classement')
def classement():
    classement = db.session.query(
        Benevole.nom, Benevole.prenom, db.func.count(Inscription.id).label('nb_missions')
    ).join(Inscription).group_by(Benevole.id).order_by(db.desc('nb_missions')).all()

    return render_template('classement.html', classement=classement)


In [12]:
from datetime import datetime, timedelta
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# Initialize Flask app
app = Flask(__name__)

# Configure database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///your_database.db'  # Replace with your database URI
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# Define models
class Benevole(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    prenom = db.Column(db.String(100))
    email = db.Column(db.String(120))

class Inscription(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    benevole_id = db.Column(db.Integer, db.ForeignKey('benevole.id'))
    date_inscription = db.Column(db.DateTime)

# Function to send email (implement according to your email service)
def envoyer_email(email, subject, message):
    # Implement your email sending logic here
    pass

# Fonction pour notifier les bénévoles inactifs
def notifier_inactifs():
    seuil = datetime.utcnow() - timedelta(days=60)
    inactifs = db.session.query(Benevole).outerjoin(Inscription, Benevole.id == Inscription.benevole_id) \
        .filter((Inscription.date_inscription == None) | (Inscription.date_inscription < seuil)).all()

    for benevole in inactifs:
        envoyer_email(benevole.email, "🌟 Rappel : Engagez-vous dans une mission !",
                      f"Bonjour {benevole.prenom}, nous avons remarqué que vous n'avez pas participé à une mission récemment. "
                      "Rejoignez une mission dès maintenant !")

# Lancer l'application
if __name__ == '__main__':
    with app.app_context():
        db.create_all()
        #importer_benevoles()  # Importer les bénévoles dès le démarrage de l'application
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

SystemExit: 1

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:

# Fonction pour notifier les bénévoles inactifs
def notifier_inactifs():
    seuil = datetime.utcnow() - timedelta(days=60)
    inactifs = db.session.query(Benevole).outerjoin(Inscription, Benevole.id == Inscription.benevole_id) \
        .filter((Inscription.date_inscription == None) | (Inscription.date_inscription < seuil)).all()

    for benevole in inactifs:
        envoyer_email(benevole.email, "🌟 Rappel : Engagez-vous dans une mission !",
                      f"Bonjour {benevole.prenom}, nous avons remarqué que vous n'avez pas participé à une mission récemment. "
                      "Rejoignez une mission dès maintenant !")

# Lancer l'application
if __name__ == '__main__':
    with app.app_context():
        db.create_all()
        #importer_benevoles()  # Importer les bénévoles dès le démarrage de l'application
    app.run(debug=True)

 * Serving Flask app 'COP1_du_13'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

SystemExit: 1

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
print("📂 Importation des bénévoles terminée !")

📂 Importation des bénévoles terminée !


In [ ]:
# Lancer l'application
if __name__ == '__main__':
    with app.app_context():
        db.create_all()
        importer_benevoles()  # Importer les bénévoles dès le démarrage de l'application
    app.run(debug=True)